<a href="https://colab.research.google.com/github/MichalRyszardWojcik/hello-world/blob/master/2020_08_31_grawszachy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
! pip install -q -U trax
import trax

In [23]:
import chess
import chess.pgn
import chess.svg

In [17]:
def move2token(move): return 64*move.from_square + move.to_square

def game2tokens(game):
  tokens = []
  line = game.main_line()
  for move in line: tokens.append(move2token(move))
  return tokens

def tokens2board(tokens):
  board = chess.Board()
  for x in tokens:
    from_square = x//64
    to_square = x % 64
    move = chess.Move(from_square,to_square)
    board.push(move)
  return board

In [ ]:
!wget https://www.pgnmentor.com/players/Bronstein.zip

In [ ]:
!unzip Bronstein.zip

In [112]:
pgn = open('Bronstein.pgn', "r")

In [ ]:
ile = 0
finished = False
tokenized_games = []
while (not finished):
  game = chess.pgn.read_game(pgn)
  if (game != None):
    ile += 1
    tokenized_games.append(game2tokens(game))
  else:
    finished = True

print(ile)  # should be 1930 games
from IPython.display import HTML
for i in range(7):
  board = tokens2board(tokenized_games[i])
  display(HTML('<div style="display:inline-block;width:11em;">'+chess.svg.board(board)+'</div>'))

In [180]:
def random_tokenized_game():
  # ile = len(tokenized_games)
  i = np.random.randint(0, ile)
  return tokenized_games[i]

In [364]:
def training_input(batch_size, length):
  while True:
    shape = (batch_size,length)
    games = np.zeros(shape, np.int32)
    for y in range(batch_size):
      game = random_tokenized_game()[0:length-1] # leaves a zero at the end
      for x in range( min( len(game), length ) ):
        games[y,x] = game[x]
    inputs = games
    targets = games
    loss_weights = np.ones(shape, np.int32)
    yield (inputs,targets,loss_weights)

game_length = 60 #40

trax_inputs = trax.data.inputs.Inputs(lambda _: training_input(batch_size=128, length=game_length))

In [365]:
def Chess_transformer_lm(mode='train'):
  return trax.models.TransformerLM(  
          d_model=128,
          d_ff=256,
          n_heads=2,
          n_layers=2, 
          vocab_size= g_vocab_size,
          mode=mode)

In [366]:
output_dir = os.path.expanduser('~/train_dir/')
!rm -f ~/train_dir/model.pkl.gz  # Remove old model.

# Train tiny model with Trainer.
trainer = trax.supervised.Trainer(
    model=Chess_transformer_lm,
    loss_fn=trax.layers.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adafactor,  # Change optimizer params here.
    lr_schedule=trax.lr.warmup_and_rsqrt_decay(400, 0.01),  # Change lr schedule here.
    inputs=trax_inputs,
    output_dir=output_dir)

n_epochs = 10
train_steps = 500
eval_steps = 2
for _ in range(n_epochs):
  trainer.train_epoch(train_steps, eval_steps)



Step    500: Ran 500 train steps in 31.82 secs
Step    500: Evaluation
Step    500: train                   accuracy |  0.35214847
Step    500: train                       loss |  3.23293948
Step    500: train         neg_log_perplexity | -3.23293948
Step    500: train          sequence_accuracy |  0.00000000
Step    500: train weights_per_batch_per_core |  7680.00000000
Step    500: eval                    accuracy |  0.34283856
Step    500: eval                        loss |  3.27571869
Step    500: eval          neg_log_perplexity | -3.27571869
Step    500: eval           sequence_accuracy |  0.00000000
Step    500: eval  weights_per_batch_per_core |  7680.00000000
Step    500: Finished evaluation

Step   1000: Ran 500 train steps in 13.46 secs
Step   1000: Evaluation
Step   1000: train                   accuracy |  0.49915367
Step   1000: train                       loss |  2.05663395
Step   1000: train         neg_log_perplexity | -2.05663395
Step   1000: train          sequence_

In [367]:
def output(input):
  predict_model = Fibo_transformer_lm(mode='predict')
  predict_signature = trax.shapes.ShapeDtype((1,1), dtype=np.int32)
  predict_model.init_from_file(os.path.join(output_dir, "model.pkl.gz"),weights_only=True, input_signature=predict_signature)
  return trax.supervised.decoding.autoregressive_sample(predict_model, input, temperature=0.0, max_length=game_length, eos_id = 0)

In [368]:
def fullgame(firstmoves):
  for i in range(len(firstmoves)):
    uci = firstmoves[i]
    move = chess.Move.from_uci(uci)
    firstmoves[i] = move2token(move)
  input = np.array(firstmoves).reshape((1,len(firstmoves)))
  played = output(input)
  gamestart = input.flatten()
  gameplayed = played.flatten()
  game = np.concatenate((gamestart,gameplayed))
  return game

from IPython.display import Javascript

def showgame(tokens):
  board = chess.Board()
  html = ''
  for x in tokens:
    from_square = x//64
    to_square = x % 64
    move = chess.Move(from_square,to_square)
    if (x == 0):
      board.clear()
      header = 'Game Over'
    else:
      legal = move in board.legal_moves
      if legal:
        legal = 'legal'
        style = ''
      else:
        legal = 'illegal'
        style = 'background:red; color:yellow; font-weight:bold;'
      header = '<div style="'+style+'">after '+legal+' '+move.uci()+'</div>'
      board.push(move)
    style = 'display:inline-block; width:20em; text-align:center; margin-bottom: 1em;'
    html += '<div style="'+style+'">'+header+chess.svg.board(board)+'</div>'
  display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
  display(HTML(html))

In [369]:
showgame(fullgame(['f2f3','f7f6']))

<IPython.core.display.Javascript object>

In [370]:
showgame(fullgame(['g1h3']))

<IPython.core.display.Javascript object>

In [371]:
showgame(fullgame(['f2f3','e7e5','g2g4']))

<IPython.core.display.Javascript object>